In [1]:
# https://ascopubs.org/action/doSearch?field1=AllField&text1=bevacizumab&Ppub=&Ppub=&AfterYear=2015&BeforeYear=2021&pageSize=20

# 0- Connect to colab drive

In [2]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [3]:
drive_dir = '/drive/MyDrive'
data_dir = drive_dir + '/nlp_nature'

# 1- Récupération des liens vers les pages des articles.

In [4]:
! pip install beautifulsoup4==4.9.3

     |████████████████████████████████| 115 kB 5.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [5]:
import requests
from bs4 import BeautifulSoup

In [6]:
jco_url = 'https://ascopubs.org/action/doSearch?field1=AllField&text1=bevacizumab&Ppub=&Ppub=&AfterYear=2015&BeforeYear=2021&pageSize=50'
html_text = requests.get(jco_url).text

In [7]:
soup = BeautifulSoup(html_text, 'html.parser')

In [8]:
print(soup.title)

<title>[All: bevacizumab] AND [Publication Date: (01/01/2015 TO 12/31/2021)] :  Search</title>


## 1-1- Recuperation des hyperliens vers les différents articles

In [9]:
href_prefix = 'https://ascopubs.org'

In [10]:
links = []
attrs={'class':'ref nowrap'}
for link in soup.find_all('a', attrs=attrs):
  links.append(link.get('href'))

print(len(links))

50


In [11]:
print(links[0])

/doi/full/10.1200/JCO.20.01225


## 1-2- Navigation sur les pages secondaires

In [12]:
#Lecture des autres pages de liens si all_pages est à True

all_pages=True
if all_pages :
  page=2
  nb_links = 1
  while nb_links != 0 :
    url = jco_url + '&startPage=' + str(page) 
    # Lecture de la page
    soup_tmp = BeautifulSoup(requests.get(url).text, 'html.parser')
    articles_links_tmp = [ link.get('href') for link in soup_tmp.find_all('a', attrs=attrs)]
    
    nb_links = len(articles_links_tmp)
    if nb_links > 0 :
      links = [*links, *articles_links_tmp]
    #print('nb_links ', nb_links)
    page += 1

In [13]:
# On remplace la recherche d'abstract par celle d'article.
full_links = [href_prefix + link.replace('/full/', '/abs/') for link in links]

## 1-3- Creation d'un dataframe contenant les liens

In [14]:
# creation d'un csv contenant les liens
import pandas as pd
jco_link_df = pd.DataFrame(full_links, columns=['link'])

In [15]:
jco_link_df.head()

,link
0,https://ascopubs.org/doi/abs/10.1200/JCO.20.01225
1,https://ascopubs.org/doi/abs/10.1200/JOP.2015....
2,https://ascopubs.org/doi/abs/10.1200/JCO.2015....
3,https://ascopubs.org/doi/abs/10.1200/JCO.19.03158
4,https://ascopubs.org/doi/abs/10.1200/JCO.2014....


# 2- Extraction des inforamtions des  différents articles et saisie dans le dataframe


In [16]:
## Champ Titre:
# <h1 class="chaptertitle">Title</h1>
## Champ auteurs :
# <a class="entryAuthor" >Yu-Hsi Lin</a>
## Champ abstract :
# <div class="h1F1d-Abstract">
# <p>Inhibiting glycolysis remains an aspirational approach for the treatment of cancer.</p></div>


In [17]:
! pip install tdqm

  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1323 sha256=b733b5d0e3659140d1200753584d505a7c3d79060d493dcc45323dc116cdb3f0
  Stored in directory: /root/.cache/pip/wheels/c6/f0/d9/9fa5ff78c0f9d5a0a427bbbb4893c283520ddfccb885ea2205
Successfully built tdqm


In [31]:
class Jco_scrap :
   def __init__(self,link):
		  self.link=link
		  self.soup=BeautifulSoup(requests.get(link).text, 'html.parser')
    
   def get_title(self):
		  return self.soup.find('div', {'class':"publicationContentTitle"}).get_text()
    
   def get_type(self):
      type_field = self.soup.find('div', {'class':"tocHeading"}).get_text(separator=separateur).split('\n')
      return type_field[1].replace('##', '')

   def get_journal(self) :
      return self.soup.find('cite').get_text()

   def get_authors(self) :
      l_auth = []
      for l in self.soup.find_all('a', attrs={'class':'entryAuthor'}):
         l_auth.append(l.get_text())
      return ','.join(l_auth)

   def get_article(self) :
        raw_article = self.soup.find('article', class_="article")
        if raw_article != None :
           return raw_article.get_text(separator=separateur)
        else :
           return 'no article'

   def get_conclusion(self):  
      raw_conclusion = self.soup.find('div', class_="abstractSection")
      if raw_conclusion != None :
         raw_conclusion_field = raw_conclusion.get_text(separator=separateur)
         l_heads = []
         for l in raw_conclusion.find_all('div', {'class':'sectionInfo'}) :
            text = l.get_text()
            #print(text)
            if 'conclusion' in text.lower() :
               conclusion_field = raw_conclusion_field.split(f'##{text}##')[-1]
               return conclusion_field.replace('##',' ')
      else :
         return 'no conclusion'



In [ ]:
import pandas as pd
import time
from tqdm import tqdm
len_max=300
separateur='##'

start_0=time.time()

l_authors = []
l_title = []
l_type = []
l_journal = []
l_conclusion = []
l_article = []

for i in tqdm(range(len(full_links[:]))) :# enumerate(tqdm(articles_links, total=len(articles_links))) :
  article_soup = Jco_scrap(full_links[i])

  l_title.append(article_soup.get_title())

  # Recuperation type
  l_type.append(article_soup.get_type())


  # Recuperation journal
  l_journal.append(article_soup.get_journal())

  # Recuperation auteurs
  l_authors.append(article_soup.get_authors())
  
  # Recuperation abstract
  l_article.append(article_soup.get_article())

  # Recuperation conclusion
  l_conclusion.append(article_soup.get_conclusion())
  


jco_link_df['title'] = l_title
jco_link_df['journal'] = l_journal
jco_link_df['authors'] = l_authors
jco_link_df['type'] = l_type
jco_link_df['article'] = l_article
jco_link_df['conclusion'] = l_conclusion



end_0= time.time()

print('duree : ', end_0 - start_0)

 13%|█▎        | 85/657 [02:04<14:05,  1.48s/it]

In [ ]:
jco_link_df.head()

## 2-1 Sauvergarde de l'ensemble des articles

In [ ]:
jco_link_df.to_csv(data_dir + '/jco_link_df_bevacizumab_17102021.csv')